## Scala a Functional Programming approach 2: Solutions

__Exercise 1__: Implement function __map__ , which changing the internal type of an option but preserves its structure.  


In [ ]:

sealed trait Opt[+A]
case object None extends Opt[Nothing]
case class Some[+A](value: A) extends Opt[A]


object Opt {
  //Smart constructor
  def apply[A](value: A): Opt[A] = if (value == null) None else Some(value) 
  
  def isEmpty[A](value: Opt[A]): Boolean = value match {
    case Some(a) => true   
    case _ => false//case _ means everything else 
  }
  
  // Solution map
  def map[A,B](opt: Opt[A], f: A => B): Opt[B] = opt match {
    case Some(a) => Some(f(a))
    case None => None
  }
}

locally {
  import Opt._

//   map(Opt(null), (x: Int) => x + 1)
  map(None, (x: Int) => x + 1)
  map(Opt(1), (x: Int) => x + 1) // Test with integers
  map(Opt("fotis"), (x: String) => x.toUpperCase) // Test with strings

  case class Player(name: String, score: Int) // Test with product
  map(Opt(Player("fpas",1)), (p: Player) => p.copy(score = p.score + 1))
}

__Exercise 2__: Implement 

__filter__ which keeps the element of an option if a given predicate holds,

__exists__ which tests if a given predicate holds for the option,

__foreach__ which executes a side effect using the value of an option (if any) 

_Hint_: For _filter_ and _exists_ use a case clause with a predicate if expression. ( `case  ... if ...  => ...` )

In [ ]:
sealed trait Opt[+A]
case object None extends Opt[Nothing]
case class Some[+A](value: A) extends Opt[A]


object Opt {
  //Smart constructor
  def apply[A](value: A): Opt[A] = if (value == null) None else Some(value) 
  
  def isEmpty[A](value: Opt[A]): Boolean = value match {
    case Some(a) => true   
    case _ => false  //case _ means everything else 
  }
  
  def filter[A](opt: Opt[A], p: A => Boolean): Opt[A] = opt match {
    case Some(a) if p(a) => opt
    case _ => None
  }
  
  def exists[A](opt: Opt[A], p: A => Boolean): Boolean = opt match {
    case Some(a) => p(a)
    case None => false
  }
 
  def foreach[A, B](opt: Opt[A], f: A => Unit ): Unit = opt match {
    case Some(a) => f(a)
    case _ => ()
  }
}

locally {
  import Opt._

  filter(Opt(1), (v: Int) => v == 1)
  filter(Opt(1), (v: Int) => v < 0 )
  
  exists(Opt("Fotis"), (v: String) => v == "George")
  exists(Opt("Fotis"), (v: String) => v == "Fotis")
  
  foreach(Opt("Fotis"), println _ )
  foreach(None, println _ ) // Should not execute
}

__Exercise 3__:

Implement __tail__, which returns the tail of non empty list.

Implement __setHead__, which replaces the head of the list.

Implement __drop__, which drops the first n elements of the list.

In [ ]:
sealed trait Lst[+A]
case object Nil extends Lst[Nothing]
case class Cons[+A](head: A, tail: Lst[A]) extends Lst[A]

object Lst {

    def apply[A](ss: A*): Lst[A] =      
      if(ss.isEmpty) Nil
      else Cons(ss.head, apply(ss.tail: _*))
      
    def head[A](l: Lst[A]): A = l match {
       case Nil => sys.error("Invoking head on empty list.")
       case Cons(a,_) => a
    }
    
    def tail[A](l: Lst[A]): Lst[A] = l match {
        case Nil => sys.error("tail of empty list")
        case Cons(_,t) => t
    }
    
    def setHead[A](l: Lst[A], h: A): Lst[A] = l match {
        case Nil => sys.error("setHead on empty list")
        case Cons(_,t) => Cons(h,t)
    }
    
    def drop[A](l: Lst[A], n: Int): Lst[A] =
      if (n <= 0) l
      else l match {
        case Nil => Nil
        case Cons(_,t) => drop(t, n-1)
      }
}

__Exercise 4__: Implement __init__ which returns the first elements of a list without the last one.

_Note: Do you see something disturbing with your implementation?_ 

In [ ]:
sealed trait Lst[+A]
case object Nil extends Lst[Nothing]
case class Cons[+A](head: A, tail: Lst[A]) extends Lst[A]

object Lst {

    def apply[A](ss: A*): Lst[A] =      
      if(ss.isEmpty) Nil
      else Cons(ss.head, apply(ss.tail: _*))
      
    def head[A](l: Lst[A]): A = l match {
       case Nil => sys.error("Invoking head on empty list.")
       case Cons(a,_) => a
    }
    
    def init[A](l: Lst[A]): Lst[A] = l match {
      case Nil => sys.error("Invoking init on empty list")
      case Cons(_, Nil) => Nil
      case Cons(h,t) => Cons(h, init(t))
    }
    
    // There is a more efficient implementation using mutable list so that we will not reconstruct the list in every step
    // and to avoid stack overflows
    
//     def  init[A](l: Lst[A]): Lst[A] = {
//       import collection.mutable.ListBuffer
//       val buf = new ListBuffer[A]
      
//       def go(cur: Lst[A]): Lst[A] = cur match {
//         case Nil => sys.error("Invoking init on empty list")
//         case Cons(_, Nil) => Lst(buf.toList: _*)
//         case Cons(h,t) => buf += h; go(t)
//       }
//       go(l)
//     }
  
}

val list = Lst("1","2","3") 
val singleList = Lst("10")
val emptyList = Lst[String]() // Nil

Lst.init(list)
Lst.init(singleList)
try { Lst.init(emptyList) } catch { case x => x } 

__Exercise 5__: Implement function __map__ , which changing the internal type of a list but preserves its structure.

_Hint: You can use the foldLeft don't reinvent the wheel_

In [ ]:
sealed trait Lst[+A]
case object Nil extends Lst[Nothing]
case class Cons[+A](head: A, tail: Lst[A]) extends Lst[A]

object Lst {

    def apply[A](ss: A*): Lst[A] =      
      if(ss.isEmpty) Nil
      else Cons(ss.head, apply(ss.tail: _*))
      
    def head[A](l: Lst[A]): A = l match {
       case Nil => sys.error("Invoking head on empty list.")
       case Cons(a,_) => a
    }
    
    // Collapses the list to one value using a initial element and a binary operation. 
    // This is done from rigth to left (from the last element to the first)
    def foldRight[A,B](l: Lst[A], z: B)(f: (A,B) => B): B = l match {
      case Nil => z
      case Cons(x, xs) => f(x, foldRight(xs,z)(f))
    }
    
    // This is the tail recursive version of the operation above ( it is not always the same why?)
    // Collapses the list to one value using a initial element and a binary operation. 
    // This is done from rigth to left (from the last element to the first)
    @annotation.tailrec
    def foldLeft[A,B](l: Lst[A], z: B)(f: (B, A) => B): B = l match {
      case Nil => z
      case Cons(h,t) => foldLeft(t, f(z,h))(f)
    }
    

// This is a more straigth forward but inefficient implementation ( due to code replication)    
//     def map[A,B](l: Lst[A], f: A => B): Lst[B] = l match {
//       case Nil => Nil
//       case Cons(h, t) => Cons(f(h), map(t,f))
//     }

   def map[A,B](l: Lst[A], f: A => B): Lst[B] = foldRight(l, Nil:Lst[B])((h,t) => Cons(f(h),t))
}
val list = Lst(1,2,3)
Lst.map(list, (i: Int) => i.toString)
Lst.map(list, (i: Int) => i + 1)